# Proyecto A

## Grupo 2

- Diego Antinao C.
- Felipe Farías O.
- Matías Ignacio Saavedra Guerra
- Nazarena Águila M.

## Equipo Docente

**Profesor:** Harold Valenzuela Coloma  
**Auxiliar:** Leslie Cárdenas  
**Ayudantes:** Francisco Cáceres, Karen Quezada, Benjamín Rojas

## Resumen del Proyecto

El brazo robótico Katana tiene una rotación en la base, tres rotaciones adicionales a lo largo del brazo, una rotación para orientar el gripper y un gripper de dos dedos. El proyecto simula una rutina de Pick & Place en una cancha de fútbol, donde el robot recoge una pelota de una canasta elevada y la coloca en el arco contrario, evitando obstáculos mediante el algoritmo de path planning A*. Este algoritmo heurístico encuentra el camino de menor costo entre un nodo de origen y uno objetivo, utilizando una fórmula que combina el costo real y una estimación heurística. Para resolver el algoritmo, se creó una grilla invisible en el mundo de la simulación, que puede visualizarse con addUserDebugLine. Link al repo original [aquí](https://github.com/uos/katana_driver/tree/kinetic).

## Indicaciones generales

- En algunas funciones que deben completar hay líneas que dicen `raise NotImplementedError`, una vez que hagan sus funciones deben borrarlo, para que no tire error.
- Se recomienda fuertemente que printeen las variables o resultados de sus funciones para entender mejor lo que está pasando y puedan debuggear sus errores.
- El movimiento del robot debe ser lo más fluido posible, para ello pueden usar control de velocidad o posición, disponibles en la documentación.

---

## Parte 1: Modificación de `pickNplace_lib.py`

En la carpeta `tareas` se encuentra este código el cual debe ser modificado para aplicar el algoritmo de búsqueda, específicamente en la función `a_star_with_obstacles`. En este código también debe completarse la función `is_in_obstacle`, la que debe determinar si en un nodo de la grilla se encuentra un objeto en la simulación (un pato). Pueden haber varios métodos para esto, pero se recomienda usar funciones propias de Pybullet.

### Preliminares

Se hacen las importaciones necesarias para el desarrollo del proyecto.

In [9]:
# Importamos las librerias necesarias
import numpy as np
from queue import PriorityQueue
import pybullet as p
import pybullet_data
import time
# from pickNplace_lib import * # Descomentar si se quiere usar la libreria de funciones fuera del Jupiter Notebook

In [10]:
# Esta función verifica si un nodo está dentro de un obstáculo
def is_in_obstacle(node, obstacle_height, grid_size, lista_de_obstaculos, sub_grid_divisions=10):
    """Determina si un nodo está dentro de un obstáculo."""
    sub_grid_size = grid_size / sub_grid_divisions
    
    # Recorre la lista de obstáculos
    for obstacle_id in lista_de_obstaculos:
        # Obtener el AABB (Axis-Aligned Bounding Box) del obstáculo
        aabb_min, aabb_max = p.getAABB(obstacle_id)
        
        # Recorre las subdivisiones del nodo
        for i in range(sub_grid_divisions):
            for j in range(sub_grid_divisions):
                # Calcula las coordenadas del sub-nodo
                sub_node_x = node[0] + i * sub_grid_size
                sub_node_y = node[1] + j * sub_grid_size
                
                # Verifica si el sub-nodo está dentro del AABB del obstáculo
                if aabb_min[0] <= sub_node_x <= aabb_max[0] and aabb_min[1] <= sub_node_y <= aabb_max[1] and aabb_min[2] <= obstacle_height <= aabb_max[2]:
                    return True  # Si está dentro del obstáculo, retorna True
    
    return False  # Si no está dentro de ningún obstáculo, retorna False

# Esta función genera una grilla y una lista de obstáculos en el mundo
def grid_and_obstacles(grid_size, world_size, obstacles_height, lista_de_obstaculos):
    """Genera una grilla y una lista de obstáculos en el mundo.
    Esta función ya está implementada y no es necesario modificarla. """
    grid = []
    obstacles= []
    for x in np.arange(0, world_size*2, grid_size):
        for y in np.arange(-world_size, world_size, grid_size):
            if not is_in_obstacle([x, y], obstacles_height, grid_size, lista_de_obstaculos):
                grid.append((x,y))
            else:
                obstacles.append((x,y, obstacles_height))
    return grid, obstacles

# Esta función reconstruye el camino desde el nodo inicial hasta el nodo actual
def reconstruct_path(came_from, current):
    """ Reconstruye el camino desde el nodo inicial hasta el nodo actual.
    Esta función ya está implementada y no es necesario modificarla."""
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse() #invertir el camino para que vaya desde el inicio hasta el final
    return path

# Esta función obtiene los vecinos de un nodo en la grilla
def get_neighbors(node, grid, obstacles, grid_size, height_z):
    """" Obtiene los vecinos de un nodo en la grilla.
    Esta función ya está implementada y no es necesario modificarla."""
    directions = [(1, 0, 0), (0, 1, 0), (-1, 0, 0), (0, -1, 0)]  # Movimientos en 2D 
    scaled_directions = [(d[0] * grid_size, d[1] * grid_size, d[2] * grid_size) for d in directions]
    
    neighbors = []
    
    for direction in scaled_directions:
        neighbor = (node[0] + direction[0], node[1] + direction[1], height_z + direction[2])
        
        is_obstacle = False
        for obs in obstacles:
            if abs(neighbor[0] - obs[0]) < 0.09 and abs(neighbor[1] - obs[1]) < 0.09 and abs(neighbor[2] - obs[2]) < 0.09:#print("Obstáculo:", obs)
                is_obstacle = True
                break
        
        if not is_obstacle:
            for cell in grid:
                if abs(neighbor[0] - cell[0]) < 0.09 and abs(neighbor[1] - cell[1]) < 0.09 and abs(neighbor[2] - height_z) < 0.09:
                    neighbors.append(neighbor)
                    break

    return neighbors

# Esta función aplica el algoritmo de A* para encontrar el camino más corto entre dos puntos en un entorno con obstáculos
def a_star_with_obstacles(start, goal, grid, obstacles, grid_size, obstacles_height):
    """
    Aplica el algoritmo de A* para encontrar el camino más corto entre dos puntos en un entorno con obstáculos.
    
    :param start: Punto de inicio (tuple con coordenadas x, y, z).
    :param goal: Punto de objetivo (tuple con coordenadas x, y, z).
    :param grid: Grilla que representa el entorno.
    :param obstacles: Lista de IDs de los obstáculos en el entorno.
    :param grid_size: Tamaño de cada celda de la grilla.
    :param obstacles_height: Altura de los obstáculos a considerar.
    """
    
    def heuristic(node, goal):
        """Calcula la distancia heurística (Euclidiana) entre dos puntos."""
        return np.linalg.norm(np.array(node) - np.array(goal))
    
    # Cola de prioridad y conjuntos de puntuaciones
    set_abierto = PriorityQueue()
    set_abierto.put((0, start))  # Iniciamos con el nodo de inicio
    came_from = {}  # Rastrea de dónde viene cada nodo
    g_score = {start: 0}  # Costo real desde el inicio
    f_score = {start: heuristic(start, goal)}  # Estimación del costo total (g + h)

    # Bucle principal
    while not set_abierto.empty():
        current = set_abierto.get()[1]  # Obtener el nodo con el menor costo estimado
        
        # Si el nodo actual es el objetivo, reconstruir el camino
        if abs(current[0] - goal[0]) < 0.09 and abs(current[1] - goal[1]) < 0.09 and abs(current[2] - goal[2]) < 0.09:
            return reconstruct_path(came_from, current)
        
        # Obtener vecinos del nodo actual
        neighbors = get_neighbors(current, grid_size)
        
        for neighbor in neighbors:
            # Si el vecino está en un obstáculo, lo ignoramos
            if is_in_obstacle(neighbor, obstacles_height, grid_size, obstacles):
                continue
            
            # Costo tentativo desde el nodo actual hasta el vecino
            tentative_g_score = g_score[current] + np.linalg.norm(np.array(current) - np.array(neighbor))
            
            if tentative_g_score < g_score(neighbor,np.inf):
                # Este es un mejor camino, se actualizan los valores
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                set_abierto.put((neighbor,f_score[neighbor]))
    
    # Si no se encuentra el camino
    raise ValueError("No se encontró un camino")

# Esta función convierte un punto del mundo de la simulación a coordenadas de la grilla
def world_to_grid(world_point, grid_size, obstacles_height):
    """
    convierte un punto del mundo de la simulación a coordenadas de la grilla.
    De ser necesario, se puede modificar esta función, para obtener mejores resultados en la simulacion
    """
    x_world, y_world, z_world = world_point
    x_grid = x_world  / 0.1  
    x_grid = round(x_grid) * grid_size  
    
    y_grid = y_world / 0.1   
    y_grid = round(y_grid) * grid_size  
    
    z_grid = z_world
    
    return (x_grid, y_grid, z_grid)

In [11]:
# Si se ejecuta este script, se ejecutará el siguiente código
if __name__ == "__main__":
    pass

## Parte 2: Modificación de `katana_pickNplace.py`

Este es el código principal de la simulación, en donde se han importado las funciones del código anterior, mediante la línea `from picknplace_lib import *`. Crear una función que mueva el robot desde un punto de inicio hasta un punto final (en este caso desde la canasta hasta el arco del lado contrario). Para lograrlo, es recomendable revisar la documentación de PyBullet, ya que proporciona funciones útiles como `calculateInverseKinematics2`, `setJointMotorControlArray` o `resetJointState` (cinemática directa) las cuales permiten controlar el movimiento y la posición del brazo robótico. Además, hay funciones como `getJointState` y `getLinkState` que proporcionan información sobre los estados de las articulaciones y los eslabones del brazo, lo que puede ser de gran utilidad para ajustar y monitorear su movimiento. La función debe recibir como uno de sus parámetros el `endEffectorId` (en este caso, definido como `tool_frame_link`), que es un enlace imaginario en el centro del gripper, utilizado como referencia para la cinemática inversa.

El movimiento debe considerar que el robot se desplace hacia la canasta, oriente el gripper (ajustando su rotación en cualquier ángulo) y cierre el gripper (utilizando las funciones de PyBullet). Posteriormente, el robot debe trasladarse a lo largo de la cancha, evitando los obstáculos y abrir el gripper al llegar al punto objetivo. Para que el brazo se introduzca en la canasta, se pueden emplear varios enfoques: usar una función predefinida sin recurrir al algoritmo A\*, crear varios puntos intermedios para guiar el movimiento, o eventualmente utilizar A\* si se estima conveniente. No es necesario que tome ningún objeto, basta con implementar la rutina de movimiento.

### Código principal

Se define el script principal del proyecto.

In [12]:
# Conectar al cliente de simulación
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
useFixedBase = True  
p.setGravity(0, 0, -9.81)

# String a carpeta de modelos
modelosFolder = "D:/Universidad/Semestre 10/Robótica/ME5150_tutorials/modelos"

# Cargar los URDFs
planeId = p.loadURDF("plane.urdf")
robotId = p.loadURDF(f"{modelosFolder}/manipuladores/katana/katana.urdf", basePosition=[0, 0, 0], useFixedBase=useFixedBase)
football_pitch = p.loadURDF(f"{modelosFolder}/manipuladores/katana/football_pitch.urdf", basePosition=[1, 1.08, 0], useFixedBase=True)
duck2 = p.loadURDF(f"{modelosFolder}/manipuladores/katana/duck_orange.urdf", basePosition=[0.28, -0.2, 0],useFixedBase=True)
duck3 = p.loadURDF(f"{modelosFolder}/manipuladores/katana/duck_vhacd.urdf", basePosition=[0.4, 0, 0],useFixedBase=True)
duck4 = p.loadURDF(f"{modelosFolder}/manipuladores/katana/duck_orange.urdf", basePosition=[0.25, 0.18, 0],useFixedBase=True)
basket = p.loadURDF(f"{modelosFolder}/manipuladores/katana/basket.urdf", basePosition=[0.1, -0.3, 0.15],useFixedBase=True)

# Rotar la canasta
angle= -0.785398
orientation= p.getQuaternionFromEuler([angle, 0, 0])
p.resetBasePositionAndOrientation(basket, [0.1, -0.3, 0.15], orientation)

#TODO: Obtener el número de articulaciones y filtrar las móviles
num_joints = p.getNumJoints(robotId) 
movable_joints = []
gripper_joints = []

#filtrar las articulaciones móviles 
for i in range(num_joints):
    joint_info = p.getJointInfo(robotId, i)
    
    # Filtrar articulaciones móviles (revolute o prismatic)
    if joint_info[2] == p.JOINT_REVOLUTE or joint_info[2] == p.JOINT_PRISMATIC:
        movable_joints.append(i)
    
    # Identificar los joints del gripper
    if "finger" in str(joint_info[12]).lower():  # Si el nombre del joint contiene "finger"
        gripper_joints.append(i)

print("Articulaciones móviles:", movable_joints)
print("Articulaciones del gripper:", gripper_joints)


lista_de_obstaculos = [duck2, duck3, duck4] 
basket_obstacle = [basket]  
grid_size = 0.06 #en terminos de la grilla
world_size = 0.35 #en terminos de la grilla
obstacles_height = 0 #en terminos del mundo
tool_frame_link = 8

print(lista_de_obstaculos)

# Obtener los índices de los links de los dedos del gripper
grid, obstaculos= grid_and_obstacles(grid_size, world_size, obstacles_height, lista_de_obstaculos)


print("obstaculos",obstaculos)


""""definir la función de movimiento que recibe las posiciones a las 
que debe moverse el robot, entre otros inputs, como robotId o endEffectorId"""


#mod 
def mover_robot(robotId, endEffectorId, path, tool_frame_link, grip_coordinates, movable_joints):
    """Mueve el robot a lo largo de la trayectoria calculada por A*."""
    for punto in path:
        # Calcular la cinemática inversa para obtener las posiciones de las articulaciones
        joint_positions = p.calculateInverseKinematics(robotId, endEffectorId, punto, lowerLimits=lower_l, upperLimits=upper_l, jointRanges=joint_ranges, restPoses=pos_descanso)

        # Mover las articulaciones del brazo a las posiciones calculadas
        p.setJointMotorControlArray(robotId, movable_joints, p.POSITION_CONTROL, targetPositions=joint_positions)

        # Mover el gripper a las coordenadas definidas
        grip_joint_positions = p.calculateInverseKinematics(robotId, tool_frame_link, grip_coordinates)
        p.setJointMotorControlArray(robotId, gripper_joints, p.POSITION_CONTROL, targetPositions=grip_joint_positions)

        # # Avanzar la simulación un paso
        # p.stepSimulation()
        # time.sleep(1 / 240)  # Ajustar la velocidad de la simulación

    print("El robot ha completado el movimiento.")


start_mundo = [0.4, -0.6, obstacles_height] #se usan las coordenadas del mundo en 3D, elegir las coordenadas de inicio y goal
goal_mundo = [0.35, 0.36, obstacles_height]
grip_coordinates= [0.19, -0.31, 0.2] 
start=world_to_grid(start_mundo, grid_size, obstacles_height) #esta función se puede modificar en picknplace_lib.py
goal=world_to_grid(goal_mundo, grid_size, obstacles_height)

# print('start:',start)
# print('start:',is_in_obstacle(start, obstacles_height, grid_size, obstaculos))
# print('goal:',goal)
# print('goal:',is_in_obstacle(goal, obstacles_height, grid_size, obstaculos))
# """path es el camino que recorre en la cancha evitando obstáculos"""

path = a_star_with_obstacles(start, goal, grid, obstaculos, grid_size, obstacles_height)

""" se definen los límites de las articulaciones y los rangos de las articulaciones
Esta info se encuentra en el archivo katana.urdf"""
lower_l=[-3.025528, -0.135228, -2.221804, -2.033309, -2.993240]
upper_l=[2.891097, 2.168572, 2.054223, 1.876133, 2.870985]
joint_ranges=[5.916625, 2.303800, 4.276027, 3.909442, 5.864225]
pos_descanso=[0.5, 0.9, -0.5, -1.7]


time.sleep(5) # esperar 5 segundos antes de empezar a mover el robot
path_index = 0




while True:
    #TODO: implementar el movimiento del robot en la simulacion
    mover_robot(robotId, tool_frame_link, path, tool_frame_link, grip_coordinates, movable_joints)
    p.stepSimulation()
    time.sleep(1 / 240) # se puede ajustar la velocidad de la simulación

error: Cannot load URDF file.